In [6]:
from __future__ import print_function

import os
import cv2
import numpy as np
np.random.seed(1337)
import sys

from keras.layers import Input
from keras_vggface.vggface import VGGFace

from vocabulary import Vocabulary

In [7]:
 input_tensor = Input(shape=(224, 224, 3))
vgg_model = VGGFace(input_tensor=input_tensor, include_top=False, pooling='avg')

58916864/58909280 [==============================] - 8s 0us/step


In [8]:
vgg_model.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     